# Contexto

Esse Notebook tem como intuito realizar a análise descritiva das vendas de uma loja fictícia MultiStore. A loja contratou 
para criar um processo de Business Intelligence (BI) para as suas vendas. Essa é uma base única com muitos dados a serem explorados

# Bibliotecas
Para a primeira parte serão importadas apenas essas bibliotecas, mas caso haja a necessidade mais serão adicionadas durante a desenvoltura do notebook.

In [2]:
import pandas as pd
import numpy as np
import math
# import 

# Importando a base de dados

In [53]:
df = pd.read_excel(r'C:\Users\Andre\Analyses\stage_MultiStore.xlsx')

In [54]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Customer Age,Customer Birthday,Customer State,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,42682,42685,Second Class,CG-12520,Claire Gute,47,01-26,Active,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,42682,42685,Second Class,CG-12520,Claire Gute,29,06-09,Active,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,42533,42537,Second Class,DV-13045,Darrin Van Huff,20,10-15,Active,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,42288,42295,Standard Class,SO-20335,Sean O'Donnell,57,06-07,Active,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,42288,42295,Standard Class,SO-20335,Sean O'Donnell,65,08-17,Active,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Row ID               9994 non-null   int64  
 1   Order ID             9994 non-null   object 
 2   Order Date           9994 non-null   int64  
 3   Ship Date            9994 non-null   int64  
 4   Ship Mode            9994 non-null   object 
 5   Customer ID          9994 non-null   object 
 6   Customer Name        9994 non-null   object 
 7   Customer Age         9994 non-null   int64  
 8   Customer Birthday    9994 non-null   object 
 9   Customer State       9994 non-null   object 
 10  Segment              9994 non-null   object 
 11  Country              9994 non-null   object 
 12  City                 9994 non-null   object 
 13  State                9994 non-null   object 
 14  Regional Manager ID  9994 non-null   object 
 15  Regional Manager     9994 non-null   o

In [56]:
x = df.duplicated().sum()
print(f'Existem {x} dados duplicados na base')

Existem 0 dados duplicados na base


In [57]:
# Entendendo quantas idades cada customer tem...

df_count_age = df.groupby("Customer Name")[['Customer Age','Customer Birthday']].count()
df_count_age_filtered = df_count_age[df_count_age['Customer Age'] <= 1]
df_count_age_filtered

,Customer Age,Customer Birthday
Customer Name,,
Anthony O'Donnell,1,1
Carl Jackson,1,1
Jocasta Rupert,1,1
Lela Donovan,1,1
Ricardo Emerson,1,1


## Primeiras observações...
A coluna de idade parece não fazer sentido, assim como a data de nascimento. Já foi possível ver isso com Claire Gute, onde se tem uma idade de 47 e 29 anos e assim como a data de nascimento, também não faz sentido. Logo para a análise continuar já irei dropar essas duas colunas. \
Não tem dados nulos na base, assim como não tem dados duplicados e como é uma base dimensão com ID, então não irei dropar a coluna para checar essa informação.

In [58]:
df = df.drop(columns= ["Customer Age", "Customer Birthday"])

### Order Date e Ship Date 
Esses dois campos são representações de datas no formato numérico, então vamos transforma-lá em formato de data

In [60]:
df['Order Date'] = pd.to_datetime(df['Order Date'], origin='1899-12-30', unit='D')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], origin='1899-12-30', unit='D')

In [61]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Customer State,Segment,Country,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Active,Consumer,United States,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Active,Consumer,United States,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Active,Corporate,United States,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Active,Consumer,United States,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Active,Consumer,United States,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


# Vamos analisar u